In [1]:
import numpy as np
import torch
from utils.datasets import create_dataloader
from utils.losses import YOLOLoss
from models.yolo import Model
import matplotlib.pyplot as plt

import smdebug.pytorch as smd
from smdebug.core.reduction_config import ReductionConfig
from smdebug.core.save_config import SaveConfig
from smdebug.core.collection import CollectionKeys

import os
from pathlib import Path
import shutil
import yaml

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2022-07-11 17:14:09.566 ip-172-31-4-88:26745 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-07-11 17:14:09.617 ip-172-31-4-88:26745 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.


In [2]:
model = Model('./models/yolov4-p5.yaml').to('cuda')


                 from  n    params  module                                  arguments                     
  0                -1  1       928  models.common.Conv                      [3, 32, 3, 1]                 
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     19904  models.common.BottleneckCSP             [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  1    161152  models.common.BottleneckCSP             [128, 128, 3]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  1   2614016  models.common.BottleneckCSP             [256, 256, 15]                
  7                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]              
  8                -1  1  10438144  

In [3]:
model

Model(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): MishCuda()
    )
    (1): Conv(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): MishCuda()
    )
    (2): BottleneckCSP(
      (cv1): Conv(
        (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): MishCuda()
      )
      (cv2): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (cv3): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (cv4): Conv(
        (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): 

In [3]:
with open('/workspace/desktop/ScaledYOLOv4/data/hyp.scratch.yaml') as f:
    hyp = yaml.load(f, Loader=yaml.FullLoader)

In [4]:
model.hyp = hyp
model.nc = 80
model.gr = 1.0

In [5]:
output_path = '/workspace/desktop/debugger/'
if Path(output_path).exists():
    shutil.rmtree(output_path)

In [6]:
module = list(model.model.modules())[0][0]

In [7]:
'''save_config = SaveConfig(save_interval=5)

smd_hook = smd.Hook(out_dir='/workspace/desktop/debugger/',
                    export_tensorboard=True,
                    include_regex=['model.0.conv_input_0'],
                    save_config=save_config)'''

"save_config = SaveConfig(save_interval=5)\n\nsmd_hook = smd.Hook(out_dir='/workspace/desktop/debugger/',\n                    export_tensorboard=True,\n                    include_regex=['model.0.conv_input_0'],\n                    save_config=save_config)"

In [8]:
# smd_hook.register_module(model)

In [9]:
train_data = "/workspace/desktop/data/coco/yolo/coco/train2017.txt"

In [10]:
dataloader, dataset = create_dataloader(train_data, 640, 4, 32)

Scanning labels /workspace/desktop/data/coco/yolo/coco/labels/train2017.cache (117266 found, 0 missing, 1021 empty, 0 duplicate, for 118287 images): 100%|██████████| 118287/118287 [00:07<00:00, 14796.60it/s]


In [11]:
dataloader_iter = iter(dataloader)

In [12]:
opt = torch.optim.Adam(model.parameters(), lr=0.00001)

In [13]:
yolo_loss = YOLOLoss(model)

In [14]:
_ = model.train()

In [21]:
for i in range(100):
    opt.zero_grad()
    imgs, targets, paths, img_info = next(dataloader_iter)
    imgs = imgs.to('cuda', non_blocking=True).float() / 255.0
    pred = model(imgs)
    loss, loss_items = yolo_loss(pred, targets.to('cuda'))
    loss.backward()
    opt.step()
    print(loss)

tensor([0.90377], device='cuda:0', grad_fn=<MulBackward0>)
tensor([0.89672], device='cuda:0', grad_fn=<MulBackward0>)
tensor([0.95175], device='cuda:0', grad_fn=<MulBackward0>)
tensor([0.91755], device='cuda:0', grad_fn=<MulBackward0>)
tensor([1.04090], device='cuda:0', grad_fn=<MulBackward0>)
tensor([0.87287], device='cuda:0', grad_fn=<MulBackward0>)
tensor([0.96092], device='cuda:0', grad_fn=<MulBackward0>)
tensor([0.90015], device='cuda:0', grad_fn=<MulBackward0>)
tensor([1.04636], device='cuda:0', grad_fn=<MulBackward0>)
tensor([1.03986], device='cuda:0', grad_fn=<MulBackward0>)
tensor([1.01087], device='cuda:0', grad_fn=<MulBackward0>)
tensor([0.85827], device='cuda:0', grad_fn=<MulBackward0>)
tensor([0.93020], device='cuda:0', grad_fn=<MulBackward0>)
tensor([0.96399], device='cuda:0', grad_fn=<MulBackward0>)
tensor([1.05684], device='cuda:0', grad_fn=<MulBackward0>)
tensor([0.88667], device='cuda:0', grad_fn=<MulBackward0>)
tensor([0.93792], device='cuda:0', grad_fn=<MulBackward0

In [16]:
loss

tensor([0.99044], device='cuda:0', grad_fn=<MulBackward0>)

In [17]:
loss_items

tensor([0.10774, 0.03521, 0.10465, 0.24761], device='cuda:0')

In [55]:
yolo_loss = YOLOLoss(model)

In [56]:
yolo_loss(pred, targets.to('cuda'))

(tensor([0.99030], device='cuda:0', grad_fn=<MulBackward0>),
 tensor([0.10821, 0.03526, 0.10410, 0.24758], device='cuda:0'))

In [54]:
import math
import torch
from torch.nn.modules.loss import _Loss
import torch.nn.functional as F
from utils.torch_utils import init_seeds, is_parallel

class YOLOBCEclsLoss(_Loss):
    
    def __init__(self, weight=1.0, pos_weight=1.0, device='cuda', reduction='mean'):
        super().__init__(reduction=reduction)
        self.pos_weight = pos_weight
        self.device = device
        self.weight = weight
        self.BCEcls = torch.nn.BCEWithLogitsLoss(pos_weight=torch.Tensor([self.pos_weight])).to(self.device)
        
    def forward(self, input, target):
        assert len(input)==len(target)
        s = 3 / len(input)
        lcls = torch.zeros(1, device=self.device)
        for x, y in zip(input, target):
            lcls += self.BCEcls(x, y)
        lcls *= self.weight * s
        return lcls

class YOLOBCEobjLoss(_Loss):
    
    def __init__(self, balance, weight=1.0, pos_weight=1.0, device='cuda', reduction='mean'):
        super().__init__(reduction=reduction)
        self.balance = balance
        self.pos_weight = pos_weight
        self.device = device
        self.weight = weight
        self.BCEobj = torch.nn.BCEWithLogitsLoss(pos_weight=torch.Tensor([self.pos_weight])).to(self.device)
        
    def forward(self, input, target):
        assert len(input)==len(target)==len(self.balance)
        s = 3 / len(self.balance) * (1.4 if len(self.balance) >= 4 else 1.)
        lobj = torch.zeros(1, device=self.device)
        for x, y, w in zip(input, target, self.balance):
            lobj += self.BCEobj(x, y) * w
        lobj *= self.weight * s
        return lobj

def bbox_iou(box1, box2, x1y1x2y2=True, GIoU=False, DIoU=False, CIoU=False):
    # Returns the IoU of box1 to box2. box1 is 4, box2 is nx4
    box2 = box2.T

    # Get the coordinates of bounding boxes
    if x1y1x2y2:  # x1, y1, x2, y2 = box1
        b1_x1, b1_y1, b1_x2, b1_y2 = box1[0], box1[1], box1[2], box1[3]
        b2_x1, b2_y1, b2_x2, b2_y2 = box2[0], box2[1], box2[2], box2[3]
    else:  # transform from xywh to xyxy
        b1_x1, b1_x2 = box1[0] - box1[2] / 2, box1[0] + box1[2] / 2
        b1_y1, b1_y2 = box1[1] - box1[3] / 2, box1[1] + box1[3] / 2
        b2_x1, b2_x2 = box2[0] - box2[2] / 2, box2[0] + box2[2] / 2
        b2_y1, b2_y2 = box2[1] - box2[3] / 2, box2[1] + box2[3] / 2

    # Intersection area
    inter = (torch.min(b1_x2, b2_x2) - torch.max(b1_x1, b2_x1)).clamp(0) * \
            (torch.min(b1_y2, b2_y2) - torch.max(b1_y1, b2_y1)).clamp(0)

    # Union Area
    w1, h1 = b1_x2 - b1_x1, b1_y2 - b1_y1
    w2, h2 = b2_x2 - b2_x1, b2_y2 - b2_y1
    union = (w1 * h1 + 1e-16) + w2 * h2 - inter

    iou = inter / union  # iou
    if GIoU or DIoU or CIoU:
        cw = torch.max(b1_x2, b2_x2) - torch.min(b1_x1, b2_x1)  # convex (smallest enclosing box) width
        ch = torch.max(b1_y2, b2_y2) - torch.min(b1_y1, b2_y1)  # convex height
        if GIoU:  # Generalized IoU https://arxiv.org/pdf/1902.09630.pdf
            c_area = cw * ch + 1e-16  # convex area
            return iou - (c_area - union) / c_area  # GIoU
        if DIoU or CIoU:  # Distance or Complete IoU https://arxiv.org/abs/1911.08287v1
            # convex diagonal squared
            c2 = cw ** 2 + ch ** 2 + 1e-16
            # centerpoint distance squared
            rho2 = ((b2_x1 + b2_x2) - (b1_x1 + b1_x2)) ** 2 / 4 + ((b2_y1 + b2_y2) - (b1_y1 + b1_y2)) ** 2 / 4
            if DIoU:
                return iou - rho2 / c2  # DIoU
            elif CIoU:  # https://github.com/Zzh-tju/DIoU-SSD-pytorch/blob/master/utils/box/box_utils.py#L47
                v = (4 / math.pi ** 2) * torch.pow(torch.atan(w2 / h2) - torch.atan(w1 / h1), 2)
                with torch.no_grad():
                    alpha = v / (1 - iou + v + 1e-16)
                return iou - (rho2 / c2 + v * alpha)  # CIoU

    return iou

class YOLOGIoULoss(_Loss):
    
    def __init__(self, weight = 1.0, device='cuda'):
        super().__init__()
        self.weight = weight
        self.device = device
        
    def forward(self, box_ious):
        np = len(box_ious)
        s = 3 / np
        lbox = torch.zeros(1, device=self.device)
        for iou in box_ious:
            lbox += (1.0 - iou).mean() 
        lbox *= self.weight * s
        return lbox
    
def build_targets(p, targets, model):
    # Build targets for compute_loss(), input targets(image,class,x,y,w,h)
    det = model.module.model[-1] if is_parallel(model) else model.model[-1]  # Detect() module
    na, nt = det.na, targets.shape[0]  # number of anchors, targets
    tcls, tbox, indices, anch = [], [], [], []
    gain = torch.ones(7, device=targets.device)  # normalized to gridspace gain
    ai = torch.arange(na, device=targets.device).float().view(na, 1).repeat(1, nt)  # same as .repeat_interleave(nt)
    targets = torch.cat((targets.repeat(na, 1, 1), ai[:, :, None]), 2)  # append anchor indices

    g = 0.5  # bias
    off = torch.tensor([[0, 0],
                        [1, 0], [0, 1], [-1, 0], [0, -1],  # j,k,l,m
                        # [1, 1], [1, -1], [-1, 1], [-1, -1],  # jk,jm,lk,lm
                        ], device=targets.device).float() * g  # offsets

    for i in range(det.nl):
        anchors = det.anchors[i]
        gain[2:6] = torch.tensor(p[i].shape)[[3, 2, 3, 2]]  # xyxy gain

        # Match targets to anchors
        t = targets * gain
        if nt:
            # Matches
            r = t[:, :, 4:6] / anchors[:, None]  # wh ratio
            j = torch.max(r, 1. / r).max(2)[0] < model.hyp['anchor_t']  # compare
            # j = wh_iou(anchors, t[:, 4:6]) > model.hyp['iou_t']  # iou(3,n)=wh_iou(anchors(3,2), gwh(n,2))
            t = t[j]  # filter

            # Offsets
            gxy = t[:, 2:4]  # grid xy
            gxi = gain[[2, 3]] - gxy  # inverse
            j, k = ((gxy % 1. < g) & (gxy > 1.)).T
            l, m = ((gxi % 1. < g) & (gxi > 1.)).T
            j = torch.stack((torch.ones_like(j), j, k, l, m))
            t = t.repeat((5, 1, 1))[j]
            offsets = (torch.zeros_like(gxy)[None] + off[:, None])[j]
        else:
            t = targets[0]
            offsets = 0

        # Define
        b, c = t[:, :2].long().T  # image, class
        gxy = t[:, 2:4]  # grid xy
        gwh = t[:, 4:6]  # grid wh
        gij = (gxy - offsets).long()
        gi, gj = gij.T  # grid xy indices

        # Append
        a = t[:, 6].long()  # anchor indices
        indices.append((b, a, gj.clamp_(0, gain[3]), gi.clamp_(0, gain[2])))  # image, anchor, grid indices
        tbox.append(torch.cat((gxy - gij, gwh), 1))  # box
        anch.append(anchors[a])  # anchors
        tcls.append(c)  # class

    return tcls, tbox, indices, anch

def smooth_BCE(eps=0.1):  # https://github.com/ultralytics/yolov3/issues/238#issuecomment-598028441
    # return positive, negative label smoothing BCE targets
    return 1.0 - 0.5 * eps, 0.5 * eps
    
class YOLOLoss(_Loss):
    
    def __init__(self, model, balance=[4.0, 1.0, 0.4]):
        super().__init__()
        self.device = next(model.parameters()).device
        self.balance = balance
        self.model = model
        self.cp, self.cn = smooth_BCE(eps=0.0)
        self.cls_loss = YOLOBCEclsLoss(weight=model.hyp['cls'], 
                                       pos_weight=model.hyp['cls_pw'], 
                                       device=self.device, 
                                       reduction='mean')
        self.obj_loss = YOLOBCEobjLoss(self.balance, 
                                       weight=model.hyp['obj'], 
                                       pos_weight=model.hyp['obj_pw'], 
                                       device=self.device, 
                                       reduction='mean')
        self.GIoU_loss = YOLOGIoULoss(weight=model.hyp['giou'],
                                      device=self.device)
        
    def forward(self, inputs, targets):
        class_inputs = []
        obj_inputs = []
        box_ious = []
        class_targets = []
        obj_targets = []
        tcls, tbox, indices, anchors = build_targets(inputs, targets, self.model)
        nt = 0
        for i, pi in enumerate(inputs):
            b, a, gj, gi = indices[i]
            tobj = torch.zeros_like(pi[..., 0], device=self.device)
            n = b.shape[0]
            if n:
                nt += n
                ps = pi[b, a, gj, gi]
                
                # Regression
                pxy = ps[:, :2].sigmoid() * 2. - 0.5
                pwh = (ps[:, 2:4].sigmoid() * 2) ** 2 * anchors[i]
                pbox = torch.cat((pxy, pwh), 1).to(self.device)  # predicted box
                giou = bbox_iou(pbox.T, tbox[i], x1y1x2y2=False, CIoU=True)  # giou(prediction, target)
                box_ious.append(giou)
                tobj[b, a, gj, gi] = (1.0 - model.gr) + model.gr * giou.detach().clamp(0).type(tobj.dtype)  # giou ratio    
                obj_inputs.append(pi[..., 4])
                obj_targets.append(tobj)
                if model.nc > 1:
                    t = torch.full_like(ps[:, 5:], self.cn, device=self.device)  # targets
                    t[range(n), tcls[i]] = self.cp
                    class_inputs.append(ps[:, 5:])
                    class_targets.append(t)
        lbox = self.GIoU_loss(box_ious)
        lobj = self.obj_loss(obj_inputs, obj_targets)
        lcls = self.cls_loss(class_inputs, class_targets) if model.nc > 1 else torch.zeros(1, device=self.device)
        bs = tobj.shape[0]
        loss = lbox + lobj + lcls
        return loss * bs, torch.cat((lbox, lobj, lcls, loss)).detach()

In [53]:
len(pred)

3

In [50]:
model.hyp['cls_pw']

1.0

In [51]:
smooth_BCE(eps=0.0)

(1.0, 0.0)

In [22]:
import torch.optim as optim

In [29]:
optim.Adam

torch.optim.adam.Adam

In [28]:
getattr(optim, 'Adam')

torch.optim.adam.Adam